# LSTM network for generating text

In [ ]:
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing import sequence

In [ ]:
from google.colab import files
files.upload()

In [ ]:

# load ascii text and covert to lowercase
filename = "wonderland.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()
print(type(raw_text))


# Cleaning the data

In [ ]:
import string
translator = str.maketrans('', '', string.punctuation)
def remove_puncts(input_string):
    return input_string.translate(translator)
  
clean_word = remove_puncts(raw_text)  
clean_word = clean_word.strip()
clean_word = clean_word.replace(u'\ufeff', '')
clean_word = clean_word.replace("\n", "")




# create mapping of unique chars to integers, and a reverse mapping

In [ ]:

chars = sorted(list(set(clean_word)))

char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(chars)

# summarize the loaded data

In [ ]:
n_chars = len(clean_word)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

# prepare the dataset of input to output pairs encoded as integers

In [ ]:


seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = clean_word[i:i + seq_length]
	seq_out = clean_word[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
 
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)


In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# Adding Pre-Padded Sequence

In [ ]:


# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
X  = sequence.pad_sequences(X, maxlen=100)




In [ ]:
X.shape

# define the LSTM model

In [ ]:

model = Sequential()
model.add(Dropout(0.1))
model.add(LSTM(256, input_shape=(1, 100), return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(256))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])





In [ ]:
X.shape[2]

# define the checkpoint

In [ ]:

filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]


# fit the model

In [ ]:
model.fit(X, y, epochs=100, batch_size=128, callbacks=callbacks_list)

# pick a random seed

In [ ]:

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print( "Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters

In [ ]:

for i in range(500):
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")
